In [1]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import ChatOllama  # or whatever LLM provider you're using

# Initialize your LLM - adjust model name and add your API key as needed
llm = ChatOllama(model="qwen3:1.7b", temperature=0.3, timeout=30)



In [2]:
# Simple test: create a basic prompt and chain
test_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in exactly 2 sentences."
)

# Create a simple chain
simple_chain = test_prompt | llm | StrOutputParser()

# Test it
result = simple_chain.invoke({"topic": "machine learning"})
print(result)

<think>
Okay, the user wants me to explain machine learning in exactly two sentences. Let me start by recalling the basic definition. Machine learning is a subset of artificial intelligence where systems learn from data without explicit programming. 

Now, I need to make sure it's concise. The first sentence covers the core concept. The second sentence should highlight the key aspect, maybe the learning process and the use of data. I should avoid technical jargon but keep it accurate. Let me check if "without explicit programming" is correct. Yes, that's right. Also, mention that the system improves over time by learning patterns from data. That's a good point. 

Wait, the user might be looking for a simple yet comprehensive explanation. Let me structure it as: first sentence defines it, second sentence explains how it works. Make sure it's two sentences and exactly two. No markdown, just plain text. Alright, that should do it.
</think>

Machine learning is a subset of artificial intel

In [3]:
# Step 1 of our pipeline: Code Analysis
analysis_prompt = PromptTemplate(
    input_variables=["code"],
    template="""Analyze this code and tell me:
1. What it does (brief)
2. Any obvious inefficiencies

Code:{code}. 
Keep your response short and direct.""")



In [4]:
# Create the analysis chain
analysis_chain = analysis_prompt | llm | StrOutputParser()

# Test with a simple inefficient example
test_code = """
def find_max(numbers):
    max_val = numbers[0]
    for i in range(len(numbers)):
        if numbers[i] > max_val:
            max_val = numbers[i]
    return max_val
"""

result = analysis_chain.invoke({"code": test_code})
print("ANALYSIS RESULT:")
print(result)

ANALYSIS RESULT:
<think>
Okay, let's see. The user provided a Python function called find_max that takes a list of numbers and returns the maximum value. The code starts by setting max_val to the first element of the list. Then it loops through each element using a for loop. If the current number is larger than the current max_val, it updates max_val. Finally, it returns the max_val.

First, the question is asking for two things: what the code does and any obvious inefficiencies. Let's break it down.

What does the code do? Well, it's a straightforward implementation of finding the maximum in a list. It initializes the max_val with the first element, then iterates through each element, updating max_val if a larger one is found. So it's basically the standard approach for finding the maximum in a list. So the first part is clear.

Now, inefficiencies. The main thing here might be the use of a for loop. Wait, the code uses a for loop with range(len(numbers)), which is okay. But in Python

In [5]:
# Helper function to clean the output (remove <think> tags)
def clean_output(text):
    # Remove <think>...</think> blocks
    import re
    cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    return cleaned.strip()

# Step 2: Create an optimization prompt that uses the analysis
optimization_prompt = PromptTemplate(
    input_variables=["code", "analysis"],
    template="""Based on this analysis, rewrite the code to be more efficient: 
    Original code: 
    {code}
    Analysis:
    {analysis}

    Provide only the improved code with a brief comment explaining the change."""
)

# Create a two-step chain: analyze → optimize
two_step_chain = (
    {
        "code": RunnablePassthrough(),
        "analysis": analysis_chain | clean_output
    }
    | optimization_prompt 
    | llm 
    | StrOutputParser() 
    | clean_output
)

# Test the full chain
result = two_step_chain.invoke(test_code)
print("OPTIMIZED CODE:")
print(result)

OPTIMIZED CODE:
To improve efficiency, we replace the manual loop with Python's built-in `max()` function, which is optimized for performance and handles the maximum calculation in a single, efficient operation.

**Improved Code:**
```python
def find_max(numbers):
    return max(numbers)
```

**Comment:**  
The built-in `max()` function is more efficient than the manual loop, as it is implemented in C and optimized for performance. This reduces the code's complexity and ensures better execution speed.


In [8]:
# Step 1: Refactor for testability
testability_prompt = PromptTemplate(
    input_variables=["code"],
    template="""Refactor this code to make it more unit-testable by:
- Breaking large functions into smaller, focused functions
- Separating I/O from business logic
- Making functions pure (same input = same output) where possible
- Removing hard-coded values

Original code:
{code}

Provide the refactored code with brief comments explaining the changes."""
)

# Test with a more complex example that needs refactoring
complex_test_code = """
def calculate_total(price, tax_rate=0.08):
    tax = price * tax_rate
    total = price + tax
    print(f"Total with tax: ${total}")
    return total
"""

from langchain_core.output_parsers import StrOutputParser

refchainparser = StrOutputParser()
chunks = []
# Test the refactoring step
refactor_chain = testability_prompt | llm | refchainparser | clean_output
print("running chain")
try: 
    async for chunk in refactor_chain.astream({"code":complex_test_code}):
        print(chunk, end="|", flush=True)    
        chunks.append(chunk)
    print("finished running")
except Exception as e:
    print(f"chain failed. {e}")

print("REFACTORED FOR TESTABILITY:")
text = ''.join(chunks)
cleaned = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
print(cleaned)

running chain
To improve the code's testability, we have made several key structural and functional changes:

---

### ✅ Refactored Code

```python
def calculate_total(price, tax_rate=0.08):
    """
    Calculate the total price including tax.

    Args:
        price (float): The base price before tax.
        tax_rate (float, optional): The tax rate to apply. Defaults to 0.08.

    Returns:
        float: The total price after applying tax.
    """
    return price * tax_rate + price

def print_total(total):
    """
    Print the total price with tax formatted as a string.

    Args:
        total (float): The calculated total price.

    Returns:
        None: The function performs side effects (printing).
    """
    print(f"Total with tax: ${total}")
```

---

### 🧩 Key Changes and Explanations

1. **Breaking Large Functions into Smaller, Focused Functions**:
   - The original `calculate_total` function was a single, large function with print statements.
   - We split it into two 

NameError: name 're' is not defined

In [9]:
# Step 2: Generate unit tests for the refactored code
test_generation_prompt = PromptTemplate(
    input_variables=["code"],
    template="""Generate comprehensive unit tests for this code using pytest.
Include:
- Test normal cases
- Test edge cases
- Test invalid inputs
- Clear test names

Code to test:
{code}

Provide complete test code that can be saved to a separate file."""
)

# Extract just the refactored code from the previous result
# (We'll need to parse this from your result above)
refactored_functions = """
def calculate_tax(price, tax_rate):
    return price * tax_rate

def calculate_total(price, tax_rate):
    return price + calculate_tax(price, tax_rate)
"""

# Generate tests
test_generation_chain = test_generation_prompt | llm | StrOutputParser() | clean_output

print("Generating unit tests...")
test_code = test_generation_chain.invoke({"code": refactored_functions})
print("GENERATED TESTS:")
print(test_code)

Generating unit tests...
GENERATED TESTS:
```python
import pytest

def calculate_tax(price, tax_rate):
    return price * tax_rate

def calculate_total(price, tax_rate):
    return price + calculate_tax(price, tax_rate)

# Test Cases

def test_calculate_tax_normal():
    """Test calculate_tax with valid positive price and tax rate."""
    assert calculate_tax(100, 0.05) == 5.0

def test_calculate_tax_edge_case():
    """Test calculate_tax with price 0 and tax rate 0.0."""
    assert calculate_tax(0, 0.0) == 0.0

def test_calculate_tax_invalid_input():
    """Test calculate_tax with non-numeric price."""
    with pytest.raises(TypeError):
        calculate_tax('100', 0.05)

def test_calculate_total_normal():
    """Test calculate_total with valid positive price and tax rate."""
    assert calculate_total(100, 0.05) == 105.0

def test_calculate_total_edge_case():
    """Test calculate_total with price 0 and tax rate 0.0."""
    assert calculate_total(0, 0.0) == 0.0

def test_calculate_to

In [24]:
# Step 3: Ask user permission to save and run tests
def extract_code_from_markdown(text):
    """Extract Python code from markdown code blocks."""
    import re
    
    # Look for ```python...``` or just ```...``` blocks
    python_pattern = r'```python\s*\n(.*?)\n```'
    general_pattern = r'```\s*\n(.*?)\n```'
    
    # Try Python-specific first
    python_match = re.search(python_pattern, text, re.DOTALL)
    if python_match:
        return python_match.group(1).strip()
    
    # Fall back to general code block
    general_match = re.search(general_pattern, text, re.DOTALL)
    if general_match:
        return general_match.group(1).strip()
    
    # If no code blocks found, return original (maybe it's already clean)
    return text.strip()

# clean our freshly produced code
test_code = extract_code_from_markdown(test_code)

filename = "test_refactored_code.py"

def ask_permission_and_run_tests(test_code, filename="test_refactored_code.py"):
    """Ask user permission to save and run the generated tests."""
    
    print(f"\n{'='*50}")
    print("🧪 READY TO SAVE AND RUN TESTS")
    print(f"{'='*50}")
    print(f"I can save these tests to '{filename}' and run them.")
    print("This will:")
    print("1. Save the test code to a .py file")
    print("2. Run pytest on the file")
    print("3. Show you the test results")
    
    # In a real interactive environment, you'd get user input
    # For now, let's assume permission granted
    permission = input("\nDo you want me to save and run these tests? (y/n): ").lower().strip()
    
    if permission == 'y' or permission == 'yes':
        return True
    else:
        print("Tests not saved or run.")
        return False


# Test the permission function
if ask_permission_and_run_tests(test_code):
    print("✅ Permission granted! Proceeding with file creation and test execution...")
    with open(filename, "w") as f:    
        # Save the test file
        f.write(test_code)
        print("📁 Test file saved as 'test/test_refactored_code.py'")
        f.close()
    
    # Run the tests (we'll add this next)
    print("🚀 Running tests...")
else:
    print("❌ Skipping test execution.")


🧪 READY TO SAVE AND RUN TESTS
I can save these tests to 'test_refactored_code.py' and run them.
This will:
1. Save the test code to a .py file
2. Run pytest on the file
3. Show you the test results



Do you want me to save and run these tests? (y/n):  y


✅ Permission granted! Proceeding with file creation and test execution...
📁 Test file saved as 'test/test_refactored_code.py'
🚀 Running tests...


In [25]:
import subprocess
import os

def run_tests_and_show_results(test_filename="test_refactored_code.py"):
    """Run the pytest tests and display results."""
    
    if not os.path.exists(test_filename):
        print(f"❌ Test file '{test_filename}' not found!")
        return False
    
    try:
        # Run pytest with verbose output
        result = await subprocess.run(
            ["python", "-m", "pytest", test_filename, "-v"], 
            capture_output=True, 
            text=True,
            timeout=30
        )
        
        print(f"\n{'='*60}")
        print("🧪 TEST RESULTS")
        print(f"{'='*60}")
        
        print("STDOUT:")
        print(result.stdout)
        
        if result.stderr:
            print("STDERR:")
            print(result.stderr)
        
        print(f"\n📊 Return code: {result.returncode}")
        
        if result.returncode == 0:
            print("✅ All tests passed!")
        else:
            print("❌ Some tests failed or there were errors.")
            

SyntaxError: incomplete input (3196246639.py, line 36)